In [3]:
import geopandas as gpd
import pandas as pd

In [4]:
stations_read = gpd.read_file("../subway_station_accessibility.geojson")

In [5]:
stations = stations_read.reindex(["complex_id", "line", "ada_status_code", "geometry"], axis=1)

In [10]:
unique_station_lines = set()
for station_line in stations['line']:
    lines = station_line.split('-')
    unique_station_lines.update(lines)

In [11]:
stations_on_lines = {}
for unique_line in unique_station_lines:
    stations_on_line = stations[stations.line.str.contains(unique_line)]
    accessible_stations_on_line = stations_on_line[stations_on_line.ada_status_code <= 2]
    stations_on_lines[unique_line] = {'all': stations_on_line, 'accessible': accessible_stations_on_line}

In [12]:
nearest_accessible_neighbors = {}
for unique_line, _stations in stations_on_lines.items():
    all_stations = _stations['all']
    accessible_stations = _stations['accessible']
    station_accessible_neighbors = {}
    for index, station in all_stations.iterrows():
        station_complex_id = station.complex_id
        station_df = all_stations[all_stations.complex_id == station_complex_id]
        accessible_neighbors = accessible_stations[accessible_stations.complex_id != station_complex_id]
        nearest_accessible = gpd.sjoin_nearest(station_df, accessible_neighbors, distance_col='distance')
        nearest_accessible_distance = nearest_accessible.iloc[0, 8]
        nearest_accessible_complex_id = nearest_accessible.iloc[0, 5]
        prev_nearest = nearest_accessible_neighbors.get( station_complex_id, 0)
        nearest_accessible_neighbors[station_complex_id] = prev_nearest if prev_nearest > nearest_accessible_distance else nearest_accessible_distance

/home/miller/Documents/GeoI/spring-2023/subway-ada-sdss/api/ada_stations/data/process-scripts/process-venv/lib64/python3.11/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/miller/Documents/GeoI/spring-2023/subway-ada-sdss/api/ada_stations/data/process-scripts/process-venv/lib64/python3.11/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
/home/miller/Documents/GeoI/spring-2023/subway-ada-sdss/api/ada_stations/data/process-scripts/process-venv/lib64/python3.11/site-packages/geopandas/array.py:344: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use

In [13]:
nearest_accessible_neighbors_df = pd.DataFrame({
    "complex_id": nearest_accessible_neighbors.keys(),
    "nearest_accessible_neighbor": nearest_accessible_neighbors.values()
})

In [14]:
nearest_accessible_neighbors_df.to_csv("subway_station_ada_gaps.csv")